In [1]:
from mycelia.shared.config import MinerConfig, ValidatorConfig
from mycelia.shared.cycle import get_validator_miner_assignment
from mycelia.miner.train import * 
import bittensor as bt 

subtensor = bt.subtensor('test')

path = "/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation/config.yaml"
config = MinerConfig.from_path(path)

PyTorch version 2.9.1 available.


/home/isabella/crucible/subnet-MoE/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rank = 2
# === set up chain worker ===
wallet, subtensor = setup_chain_worker(config)

# === mis ===
device = torch.device(f"cuda:{rank}" if torch.cuda.is_available() else "cpu")
tokenizer = get_base_tokenizer(config)

# === set up training ===
(
    model,
    global_model,
    inner_optimizer,
    outer_optimizer,
    inner_scaler,
    outer_scaler,
    scheduler,
    # start_step,
    expert_manager,
    train_dataloader,
    current_model_meta,
) = setup_training(config, rank, device, tokenizer, subtensor, wallet, current_model_meta=None)

2025-12-12 23:50:46 [info     ] rank 2: No checkpoints found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation [mycelia.shared.checkpoint]
2025-12-12 23:50:46 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_dummy'),)
2025-12-12 23:50:46 [info     ] load_expert_group_assignment - folder [mycelia.shared.expert_manager] positional_args=(PosixPath('/home/isabella/crucible/subnet-MoE/expert_groups/exp_math'),)
2025-12-12 23:50:49 [info     ] Fetching model from chain    [mycelia.shared.chain] should_download=False
2025-12-12 23:50:49 [info     ] Get base model for checkpoint [mycelia.shared.model] group_ids=[1] partial=True
2025-12-12 23:50:49 [info     ] Loading in standard mode for miner [mycelia.shared.modeling.mycelia]


The fast path is not available because one of the required library is not installed. Falling back to torch implementation. To install follow https://github.com/fla-org/flash-linear-attention#installation and https://github.com/Dao-AILab/causal-conv1d


2025-12-12 23:51:48 [info     ] Load base model for checkpoint [mycelia.shared.model]
2025-12-12 23:51:48 [info     ] rank 2: No checkpoints found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation [mycelia.shared.checkpoint]
2025-12-12 23:51:48 [info     ] rank 2: No checkpoints found in /home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation [mycelia.shared.checkpoint]
2025-12-12 23:51:48 [info     ] secondary checkpoint not found [mycelia.shared.checkpoint] secondary_ckpt_path=PosixPath('/home/isabella/crucible/subnet-MoE/checkpoints/miner/miner/hk1/foundation')
2025-12-12 23:51:48 [info     ] Tried to resume from checkpoint, but no checkpoint found. [mycelia.shared.model]
2025-12-12 23:53:30 [info     ]  rank 2 optimizer            [mycelia.miner.train]
2025-12-12 23:53:30 [info     ]  rank 2 scheduler            [mycelia.miner.train]
2025-12-12 23:53:32 [info     ] rank 2 setup_training: success! [mycelia.miner.train]


In [3]:
iter_data = iter(train_dataloader)

In [8]:
model

OptimizedModule(
  (_orig_mod): CustomQwen3NextModel(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-2): 3 x DecoderLayer(
        (linear_attn): Qwen3NextGatedDeltaNet(
          (act): SiLUActivation()
          (conv1d): Conv1d(8192, 8192, kernel_size=(4,), stride=(1,), padding=(3,), groups=8192, bias=False)
          (in_proj_qkvz): Linear(in_features=2048, out_features=12288, bias=False)
          (in_proj_ba): Linear(in_features=2048, out_features=64, bias=False)
          (norm): Qwen3NextRMSNormGated()
          (out_proj): Linear(in_features=4096, out_features=2048, bias=False)
        )
        (mlp): SparseMoeBlock(
          (gate): TopKRouter(
            (weight): Linear(in_features=2048, out_features=11, bias=False)
          )
          (experts): ModuleDict(
            (0): Qwen3NextMLP(
              (gate_proj): Linear(in_features=2048, out_features=512, bias=False)
              (up_proj): Linear(in_features=2048, out_features=512, b

In [7]:
batch = next(iter_data)

model.train()

batch_device = {}
for key in batch.keys():
    batch_device[key] = batch[key].to(device)

with torch.amp.autocast("cuda", dtype=torch.float16):
    logger.info("batch device keys", batch_device.keys())
    outputs = model(**batch_device)

    logger.info("model output", outputs)

2025-12-13 00:18:09 [info     ] batch device keys            [mycelia.miner.train] positional_args=(dict_keys(['input_ids', 'attention_mask', 'labels']),)
2025-12-13 00:18:10 [info     ] model output                 [mycelia.miner.train] positional_args=(MoeModelOutputWithPast(last_hidden_state=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan,